In [2]:
# Import everything we need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime
from datetime import datetime as dt

from sklearn.model_selection import cross_val_score, TimeSeriesSplit, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV, RidgeCV
import xgboost
from xgboost import XGBRegressor 
import lightgbm as lgb

from plotly import __version__ 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import chart_studio.plotly
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
init_notebook_mode(connected = True)

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [6]:
# Function better understanding the tables
def feature_summary(df_fa):
    print('DataFrame shape')
    print('rows:',df_fa.shape[0])
    print('cols:',df_fa.shape[1])
    col_list=['Null','Unique_Count','Data_type','Max/Min','Mean','Std','Skewness','Sample_values']
    df=pd.DataFrame(index=df_fa.columns,columns=col_list)
    df['Null']=list([len(df_fa[col][df_fa[col].isnull()]) for i,col in enumerate(df_fa.columns)])
    #df['%_Null']=list([len(df_fa[col][df_fa[col].isnull()])/df_fa.shape[0]*100 for i,col in enumerate(df_fa.columns)])
    df['Unique_Count']=list([len(df_fa[col].unique()) for i,col in enumerate(df_fa.columns)])
    df['Data_type']=list([df_fa[col].dtype for i,col in enumerate(df_fa.columns)])
    for i,col in enumerate(df_fa.columns):
        if 'float' in str(df_fa[col].dtype) or 'int' in str(df_fa[col].dtype):
            df.at[col,'Max/Min']=str(round(df_fa[col].max(),2))+'/'+str(round(df_fa[col].min(),2))
            df.at[col,'Mean']=df_fa[col].mean()
            df.at[col,'Std']=df_fa[col].std()
            df.at[col,'Skewness']=df_fa[col].skew()
        df.at[col,'Sample_values']=list(df_fa[col].unique())
           
    return(df.fillna('-'))

# Split data in a different way
def timeseries_train_test_split(X, y, test_size):
    """
        Perform train-test split with respect to time series structure
    """
    
    # get the index after which test set starts
    test_index = int(len(X)*(1-test_size))
    
    X_train = X.iloc[:test_index]
    y_train = y.iloc[:test_index]
    X_test = X.iloc[test_index:]
    y_test = y.iloc[test_index:]
    
    return X_train, X_test, y_train, y_test

# for time-series cross-validation set 5 folds 
tscv = TimeSeriesSplit(n_splits=5)

# Metric
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [9]:
# A function read data from my database and dropped some rows by some simple requirements
df = loadData('***') 

# There're some accounts stopping selling products after 2019, so I dropped as we don't need to predict these accounts
df = dropAccByRatio2019(df, 0.06) 

# Dropped account and product less than 6 records
df = dropAccPrdLessRecord(df, 6) 

In [10]:
# Modeling for a specific account and getting the metric results
def modelingFor5(Account):
    df_model = df[df['Account'] == Account]
    df_timeseries = pd.DataFrame(df_model.groupby(['Date'])['NetAmount'].sum()).sort_index(axis=0)
    
    data = pd.DataFrame(df_timeseries.NetAmount)
    data.columns = ["y"]
 
    # Drop data after 2019-06
    data = data.loc[data.index[:-1]]
    
    # Adding the lag of the target variable from 7 steps back up to 48 months ago
    for i in range(7, 48):
        data["lag_{}".format(i)] = data.y.shift(i)
        
    y = data.dropna().y
    X = data.dropna().drop(['y'], axis=1)

    # Reserve 30% of data for testing
    X_train, X_test, y_train, y_test = timeseries_train_test_split(X, y, test_size=0.3)
    
    # Scaling
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Linear Regression 
    lr = LinearRegression()
    lr.fit(X_train_scaled, y_train)
    prediction1 = lr.predict(X_test_scaled)
    error_linear = mean_absolute_percentage_error(prediction1, y_test)

    # Ridge
    ridge = RidgeCV(cv=tscv)
    ridge.fit(X_train_scaled, y_train)
    prediction2 = ridge.predict(X_test_scaled)
    error_ridge = mean_absolute_percentage_error(prediction2, y_test)
      
    # Lasso
    lasso = LassoCV(cv=tscv)
    lasso.fit(X_train_scaled, y_train)
    prediction3 = lasso.predict(X_test_scaled)
    error_lasso = mean_absolute_percentage_error(prediction3, y_test)
      
    # XGB
    xgb = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
         colsample_bynode=1, colsample_bytree=0.3, gamma=0,
         importance_type='gain', learning_rate=0.1, max_delta_step=0,
         max_depth=4, min_child_weight=1, missing=None, n_estimators=100,
         n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
         reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
         silent=None, subsample=0.5, verbosity=1)   
    xgb.fit(X_train_scaled, y_train)
    prediction4 = xgb.predict(X_test_scaled)
    error_xgb = mean_absolute_percentage_error(prediction4, y_test)

    # LightGBM
    lgb_train = lgb.Dataset(X_train_scaled, y_train)
    lgb_eval = lgb.Dataset(X_test_scaled, y_test, reference=lgb_train)
    lightgbm_params = {'boosting_type': 'gbdt', 
         'colsample_bytree': 0.65, 
         'learning_rate': 0.001, 
         'n_estimators': 20, 
         'num_leaves': 3, 
         'reg_alpha': 0.5, 
         'reg_lambda': 0.5, 
         'subsample': 0.7}
    gbm = lgb.train(lightgbm_params,lgb_train,num_boost_round=10,valid_sets=lgb_eval)
    prediction5 = gbm.predict(X_test_scaled)
    error_lightgbm = mean_absolute_percentage_error(prediction5, y_test)
    
    # Stacking
    stack_error = mean_absolute_percentage_error(prediction1*0.1+prediction2*0.075+prediction3*0.075+prediction4*0.35+prediction5*0.4, y_test)
    l1 = [error_linear, error_ridge, error_lasso, error_xgb, error_lightgbm, stack_error, 
    (error_linear+error_ridge+error_lasso+error_xgb+error_lightgbm)/5, 
    min(error_linear, error_ridge, error_lasso, error_xgb, error_lightgbm)]

    global df_final
    df_final = pd.DataFrame({Account:l1}).T
    df_final.columns = ['linear', 'ridge', 'lasso', 'xgb', 'lightgbm', 'stack_error', 'avg_error', 'min_error']

    return df_final

# Repeat modeling function for each account
def getMetics(Accountlist):
    df = modelingFor5(l1[0])
    for i in Accountlist[1:]:
        df = df.append(modelingFor5(i))
    return df

In [23]:
MetricResult = getMetics(AccountList)
MetricResult.head()

,Account,linear,ridge,lasso,xgb,lightgbm,stack_error,avg_error,min_error
0,0,4.723653,3.961553,8.215475,6.680395,9.388411,6.690356,6.593897,3.961553
1,1,12.372634,12.273017,13.172614,7.287424,7.313861,7.939729,10.483910,7.287424
2,2,9.967715,10.607925,9.849556,11.932323,8.882980,8.674076,10.248100,8.882980
3,3,9.418262,9.388396,9.542745,6.874848,10.285188,6.939145,9.101888,6.874848
4,4,10.057956,9.129196,13.594823,5.789648,7.494531,4.291907,9.213231,5.789648


In [24]:
feature_summary(MetricResult)

DataFrame shape
rows: 252
cols: 8


,Null,Unique_Count,Data_type,Max/Min,Mean,Std,Skewness,Sample_values
linear,0,252,float64,254.28/3.26,12.187474,17.721317,11.353183,"[4.723653328963506, 12.372634233085533, 9.9677..."
ridge,0,252,float64,35.9/2.09,9.930269,4.154215,2.045378,"[3.961552604153703, 12.273017139545168, 10.607..."
lasso,0,252,float64,33.71/2.68,10.045051,3.938487,1.646532,"[8.215475280377449, 13.172613945741842, 9.8495..."
xgb,0,252,float64,17.1/3.86,8.417333,2.246964,0.551185,"[6.680394652818449, 7.287423787046589, 11.9323..."
lightgbm,0,252,float64,13.89/4.06,8.485432,1.962333,0.302005,"[9.388411416733115, 7.3138612013420135, 8.8829..."
stack_error,0,252,float64,9.99/2.79,7.460815,1.768602,-0.504680,"[6.6903560992534565, 7.939729380306641, 8.6740..."
avg_error,0,252,float64,63.78/4.23,9.813111,4.390089,8.246720,"[6.593897456609244, 10.48391006135223, 10.2480..."
min_error,0,252,float64,12.44/2.09,6.821710,1.799177,0.207878,"[3.961552604153703, 7.287423787046589, 8.88297..."


In [32]:
# Plot for a specifc account with prediction, actual amount, and historical amount
def plotFor5(Account):
    df_model = df[df['Account'] == Account]
    df_timeseries = pd.DataFrame(df_model.groupby(['Date'])['NetAmount'].sum()).sort_index(axis=0)
    data = pd.DataFrame(df_timeseries.NetAmount)
    data.columns = ["y"]
 
    # Drop data after 2019-06
    data = data.loc[data.index[:-1]] 

    # Adding the lag of the target variable from 7 steps back up to 48 months ago
    for i in range(7, 48):
        data["lag_{}".format(i)] = data.y.shift(i)

    y = data.dropna().y
    X = data.dropna().drop(['y'], axis=1)
    
    # Reserve 30% of data for testing
    X_train, X_test, y_train, y_test = timeseries_train_test_split(X, y, test_size=0.3)
    # Scaling
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Linear Regression 
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    prediction1 = lr.predict(X_test)
    error_linear = mean_absolute_percentage_error(prediction1, y_test)

    # Ridge
    ridge = RidgeCV(cv=tscv)
    ridge.fit(X_train_scaled, y_train)
    prediction2 = ridge.predict(X_test_scaled)
    error_ridge = mean_absolute_percentage_error(prediction2, y_test)

    # Lasso
    lasso = LassoCV(cv=tscv)
    lasso.fit(X_train_scaled, y_train)
    prediction3 = lasso.predict(X_test_scaled)
    error_lasso = mean_absolute_percentage_error(prediction3, y_test)

    # XGB
    xgb = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
         colsample_bynode=1, colsample_bytree=0.3, gamma=0,
         importance_type='gain', learning_rate=0.1, max_delta_step=0,
         max_depth=4, min_child_weight=1, missing=None, n_estimators=100,
         n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
         reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
         silent=None, subsample=0.5, verbosity=1)          
    xgb.fit(X_train_scaled, y_train)
    prediction4 = xgb.predict(X_test_scaled)
    error_xgb = mean_absolute_percentage_error(prediction4, y_test)

    # LightGBM
    lgb_train = lgb.Dataset(X_train_scaled, y_train)
    lgb_eval = lgb.Dataset(X_test_scaled, y_test, reference=lgb_train)
    lightgbm_params = {'boosting_type': 'gbdt', 
               'colsample_bytree': 0.90, 
               'learning_rate': 0.005, 
               'n_estimators': 40, 
               'num_leaves': 6, 
               'reg_alpha': 1, 
               'reg_lambda': 1, 
               'subsample': 0.7}
    gbm = lgb.train(lightgbm_params,lgb_train,num_boost_round=10,valid_sets=lgb_eval)
    prediction5 = gbm.predict(X_test_scaled)
    error_lightgbm = mean_absolute_percentage_error(prediction5, y_test)
    
    # Prediction
    Prediction = prediction1*0.1+prediction2*0.075+prediction3*0.075+prediction4*0.35+prediction5*0.4
    
    # Plot
    max1 = max(max(Prediction), max(y_test))
    max2 = max(max1,max(y_train))
    layout = go.Layout(xaxis=dict(autorange=True),yaxis=dict(range = [0, max2*1.2]))
    fig = go.Figure(layout= layout)
    fig.add_trace(go.Scatter(y=y_test, x= list(range(13,19)),
                mode='lines',
                name='Actual', line = dict(width=4)))
    fig.add_trace(go.Scatter(y=prediction1, x= list(range(13,19)),
                mode='lines',
                name='Linear', line = dict(width=2, dash='dash')))
    fig.add_trace(go.Scatter(y=prediction2,x= list(range(13,19)),
                mode='lines',
                name='Ridge', line = dict(width=2, dash='dash')))
    fig.add_trace(go.Scatter(y=prediction3,x= list(range(13,19)),
                mode='lines',
                name='Lasso', line = dict(width=2, dash='dash')))
    fig.add_trace(go.Scatter(y=prediction4,x= list(range(13,19)),
                mode='lines',
                name='XGB', line = dict(width=2, dash='dash')))
    fig.add_trace(go.Scatter(y=prediction5,x= list(range(13,19)),
                mode='lines',
                name='Lightgbm', line = dict(width=2, dash='dash')))
    fig.add_trace(go.Scatter(y=Prediction,x= list(range(13,19)),
                mode='lines',
                name='Prediction', line = dict(width=4)))
    y_trainPlot = list(y_train)
    y_trainPlot.append(y_test[0])
    fig.add_trace(go.Scatter(y=y_trainPlot, x= list(range(0,14)),
                mode='lines',
                name='History', line = dict(width=2,dash='dash')))
    y_fake = [y_train[-1],Prediction[0]]
    fig.add_trace(go.Scatter(y=y_fake, x= list(range(12,14)),
                mode='lines',
                name='Fake', line = dict(width=2,dash='dash')))
    fig.show()

In [36]:
plotFor5(Account = 7434)

[11:09:08] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [15]:
def predictNet_PRD(Account, Product):
    # Data preprocessing
    df_model = df[df['Account'] == Account][df['Product'] == Product]
    df_timeseries = pd.DataFrame(df_model.groupby(['Date'])['NetAmount'].sum()).sort_index(axis=0)
    data = pd.DataFrame(df_timeseries.NetAmount)
    data.columns = ["y"]
    data = data.loc[data.index[:-1]]
    X_predict = pd.DataFrame(index = ['2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12'])
    data = pd.concat([data,X_predict])
    for i in range(7, 48):
        data["lag_{}".format(i)] = data.y.shift(i)
    data = data.reset_index()
    data['index'] = pd.to_datetime(data['index'])
    data['index'] = data['index'].apply(lambda x: x.strftime("%Y-%m"))
    data_train = data[data['index'] < '2019-07'].set_index('index')
    data_test = data[data['index'] >= '2019-07'].set_index('index')
    y_train = data_train.dropna().y
    X_train = data_train.dropna().drop(['y'], axis=1)    
    X_test = data_test.drop(['y'], axis=1)   
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Linear Regression 
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    prediction1 = lr.predict(X_test)
    
    # Ridge
    ridge = RidgeCV(cv=tscv)
    ridge.fit(X_train_scaled, y_train)
    prediction2 = ridge.predict(X_test_scaled)
    
    # Lasso
    lasso = LassoCV(cv=tscv)
    lasso.fit(X_train_scaled, y_train)
    prediction3 = lasso.predict(X_test_scaled)
    
    # XGB
    xgb = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
         colsample_bynode=1, colsample_bytree=0.3, gamma=0,
         importance_type='gain', learning_rate=0.1, max_delta_step=0,
         max_depth=4, min_child_weight=1, missing=None, n_estimators=100,
         n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
         reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
         silent=None, subsample=0.5, verbosity=1)          
    xgb.fit(X_train_scaled, y_train)
    prediction4 = xgb.predict(X_test_scaled)
    
    # LightGBM
    lgb_train = lgb.Dataset(X_train_scaled, y_train)
    # lgb_eval = lgb.Dataset(X_test_scaled, y_test, reference=lgb_train)
    lightgbm_params = {'boosting_type': 'gbdt', 
               'colsample_bytree': 0.90, 
               'learning_rate': 0.005, 
               'n_estimators': 40, 
               'num_leaves': 6, 
               'reg_alpha': 1, 
               'reg_lambda': 1, 
               'subsample': 0.7}
    gbm = lgb.train(lightgbm_params,lgb_train,num_boost_round=10)
    prediction5 = gbm.predict(X_test_scaled)
    
    # Stacking
    global stack_model
    stack_model = prediction1*0.1+prediction2*0.075+prediction3*0.075+prediction4*0.35+prediction5*0.4
    global final_df
    final_df = pd.DataFrame(index = ['2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12'], data = stack_model, columns = [Account]).T
    final_df['Product'] = Product
    final_df = final_df.reset_index()
    final_df.columns = ['Account', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12', 'Product']
    return final_df.set_index(['Account','Product'])

def getPredict_PRD(AccountList,ProductList):
    d1 = predictNet_PRD(AccountList[0],ProductList)
    for i in AccountList[1:]:
        try:
            d1 = d1.append(predictNet_PRD(i,ProductList))
        except:
            pass
    return d1

def getPredict_PRD2(AccountList,ProductList):
    d1 = getPredict_PRD(AccountList,ProductList[0])
    for i in ProductList[1:]:
        try:
            d1 = d1.append(getPredict_PRD(AccountList,ProductList[i]))
        except:
            pass
    return d1

In [50]:
# Final Prediction
predictResultPRD = getPredict_PRD2(AccountList,ProductList)
predictResultPRD.head(20)

,,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
Account,Product,,,,,,
0,20,4730.157627,5170.756282,4788.486828,5424.397809,5407.867831,4179.799306
1,21,432.601077,392.859806,393.054819,410.126293,394.676076,353.446963
2,22,1239.510916,1133.931007,1126.204391,1024.522220,1236.197142,986.734004
3,23,2249.486534,2414.827824,2006.095529,2162.343460,2401.872874,2390.752056
4,24,286.999568,198.064670,363.629041,402.840904,266.489535,356.220065
5,25,126.657260,168.206306,130.317548,130.408987,130.607050,145.235409
6,26,201.285151,168.841911,223.773551,178.620335,156.690365,146.032619
7,27,486.983016,369.895739,566.698872,434.506630,507.219457,396.856565
8,28,431.703721,263.057921,421.110714,310.334040,377.895357,334.763816
